In [2]:
import xrfdc
import ipywidgets as ipw
from ipywidgets import interact, interactive, fixed, interact_manual
from signal_gen import qpsk_overlay, sdr_plots, dma_timer, dict_widget

In [3]:
ol = qpsk_overlay.QpskOverlay()
signal_ip = ol.custom_cordic_pynq_0

In [4]:
ol.dac_block.MixerSettings = {
    'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
    'EventSource':    xrfdc.EVNT_SRC_IMMEDIATE,
    'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
    'Freq':           56.860, ################### Change me!
    'MixerMode':      xrfdc.MIXER_MODE_C2R,
    'MixerType':      xrfdc.MIXER_TYPE_FINE,
    'PhaseOffset':    0.0
}
ol.dac_block.UpdateEvent(xrfdc.EVENT_MIXER)

In [5]:
ol.adc_block.MixerSettings = {
    'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
    'EventSource':    xrfdc.EVNT_SRC_TILE,
    'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
    'Freq':           106.860, ################### Change me!
    'MixerMode':      xrfdc.MIXER_MODE_R2C,
    'MixerType':      xrfdc.MIXER_TYPE_FINE,
    'PhaseOffset':    0.0
}
ol.adc_block.UpdateEvent(xrfdc.EVENT_MIXER)

In [6]:
def updatefreq(freq):
    signal_ip.write(0x00,freq) #511 max?

In [ ]:
interact(updatefreq,freq=ipw.IntSlider(min=0,max=500,step=1,value=400));

interactive(children=(IntSlider(value=400, description='freq', max=500), Output()), _dom_classes=('widget-inte…

# ADC data capture

In [ ]:
fs=4000
taps = {
    'Decimated': 
      {'get': ol.qpsk_rx.get_decimated,     'get_many': ol.qpsk_rx.get_many_decimated,     'fs':1024},
    'RRCed': 
      {'get': ol.qpsk_rx.get_rrced,         'get_many': ol.qpsk_rx.get_many_rrced,         'fs':16384},
    'CoarseSynced': 
      {'get': ol.qpsk_rx.get_coarse_synced, 'get_many': ol.qpsk_rx.get_many_coarse_synced, 'fs':4096}
}

In [ ]:
tap = 'Decimated'

In [ ]:
f=taps[tap]['get']
fm=taps[tap]['get_many']
fs=taps[tap]['fs']

#f_plot = sdr_plots.IQFreqPlot(fm(), fs)
#f_dt = dma_timer.DmaTimer(f_plot.add_frame, fm, 0.3)

iq_plot = sdr_plots.IQTimePlot(fm(), fs, w=800)
iq_dt = dma_timer.DmaTimer(iq_plot.add_data, f, 0.05)

#c_plot = sdr_plots.IQConstellationPlot(fm(), plotrange=(0, len(fm())-1), fade=True)
#c_dt = dma_timer.DmaTimer(c_plot.add_data, f, 0.05)

#tab1 = ipw.Tab([ipw.VBox([iq_plot.get_widget(), ipw.HBox(iq_dt.get_widget())]),
#                ipw.VBox([f_plot.get_widget(), ipw.HBox(f_dt.get_widget())])])
#                ipw.VBox([c_plot.get_widget(), ipw.HBox(c_dt.get_widget())]),])
tab1 = ipw.Tab([ipw.VBox([iq_plot.get_widget(), ipw.HBox(iq_dt.get_widget())])])
tab1.set_title(0, 'Time domain')
#tab1.set_title(1, 'Frequency domain')
#tab1.set_title(2, 'Constellation')
tab1

In [ ]:
def vnaSlider(freq):
    freq_MHz = freq*0.125
    f_down = 50.0 + freq_MHz # [MHz]
    ol.adc_block.MixerSettings = {
    'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
    'EventSource':    xrfdc.EVNT_SRC_TILE,
    'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
    'Freq':           f_down, ################### Change me!
    'MixerMode':      xrfdc.MIXER_MODE_R2C,
    'MixerType':      xrfdc.MIXER_TYPE_FINE,
    'PhaseOffset':    0.0
    }
    ol.adc_block.UpdateEvent(xrfdc.EVENT_MIXER)
    signal_ip.write(0x00,freq)

In [ ]:
interact(vnaSlider,freq=ipw.IntSlider(min=0,max=500,step=1,value=250));

In [ ]:
iq_dt = dma_timer.DmaTimer(iq_plot.add_data, f, 0.05)

In [ ]:
iq_dt

In [ ]:
d = iq_plot.add_data

In [ ]:
d